# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457771 , -1.8541754 , -5.169583  ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038486 ,  1.1884269 ],
       [ 4.6476874 ,  8.37788   , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.48384127],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.5611653 , -5.6640916 , -1.8229512 , -9.2925    ,
         0.730283  ,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837426 , -4.1515303 ,  4.369667  , -3.0020502 ,  3.6388965 ,
        -4.341912  , -3.318711  ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43282 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809873  ,  8.42267   , -9.239023  ,  9.379146  ,  8.499882  ,
        -1.0592817 ,  3.3437858 , -7.8026104 , -0.59463334,  0.264476  ,
         5.5073943 , -4.1069794 ,  4.2890778 , -2.8172042 ,  3.615015  ,
        -4.1612997 , -3.6209633 ,  6.2185297 , -6.9460473 , -1.0828307 ,
        -5.8267713 ,  2.2258785 , -3.8601217 , -1.6974078 , -5.313417  ,
         7.5795784 ,  2.918748  ,  8.540424  ,  1.5523204 ,  1.0841808 ],
       [-2.8941858 ,  4.47419   , -4.447567  ,  2.382099  ,  1.7478831 ,
        -2.5046244 , -5.2083297 , -1.6937687 , -8.134755  ,  2.6468298 ,
        -4.3163643 ,  5.565539  , -5.7321978 , -1.7384952 , -9.344657  ,
         0.70846575,  4.43584   , -2.9008994 , -4.9486384 ,  9.695303  ,
         8.366519  , -6.247453  , -6.3494725 ,  1.9546974 ,  4.1576157 ,
        -9.167901  ,  4.607067  ,  8.788585  ,  6.864423  ,  2.2319872 ],
       [-4.6657157 , -9.558957  ,  6.657227  ,  4.4401298 ,  2.17303   ,
         2.5904047 ,  0.58000976,  6.2550364 , -8